# Practice Geocoding

## 1. Installs and Imports

In [1]:
# non-standard lib imports
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import ArcGIS
from geopy import distance
from geopy.point import Point
import geopy
import shapefile
import pandas as pd
import plotly.express as px

# standard lib
import math

## 2. Start Geocoding

In [2]:
# initialize geocoder using ArcGIS mapping systems
geocoder = ArcGIS()

Geocoded addresses return the geocoded address, and a tuple of lat, long, alt.

Sample geocoding solution:

In [3]:
str_loc1: str = '2419 Ashbury Circle Cape Coral, FL 33991'
coded1: geopy.location.Location = geocoder.geocode(str_loc1)
coded1

Location(2419 Ashbury Cir, Cape Coral, Florida, 33991, (26.62169000442219, -82.02573496942675, 0.0))

In [4]:
str_loc2: str = '2855 Gulf to Bay Blvd Clearwater FL 33759'
coded2: geopy.location.Location = geocoder.geocode(str_loc2)
coded2

Location(2855 Gulf To Bay Blvd, Clearwater, Florida, 33759, (27.95835600139496, -82.720330964178, 0.0))

Calculate distance between two addresses.

**\*Important**: geopy.distance.lonlat takes longitude first, *then* latitude.

Steps:
- Convert a geopy.location.Location into a geopy.point.Point using geopy.distance.lonlat
- Use geopy.distance.distance to calculate distance between Points

In [5]:
p1 = distance.lonlat(coded1.longitude, coded1.latitude)
p2 = distance.lonlat(coded2.longitude, coded1.latitude)
d = distance.distance(p1, p2)
d

Distance(69.17116695821905)

Returned Distance is a **straight line** and defaults to kilometers (km) but can be converted to miles.

In [6]:
print(f"Kilometers: {d.km}")
print(f"Miles: {d.miles}")

Kilometers: 69.17116695821905
Miles: 42.980970481276245


## 3. Geocode Addresses from Data File

In [2]:
target_cols = ('Location Address', 'City', 'State Code', 'Zip')
df = pd.read_csv('../data/wastewater_plants.csv', usecols=target_cols)
df.dropna(inplace=True, how='all') # remove null rows
df.sample(3)

,Location Address,City,State Code,Zip
141,1041 POND CREEK RD,STONE,KY,41567.0
146,899 RESERVOIR RD,NEW CASTLE,KY,40050.0
205,SEWER PLANT RD,CROFTON,KY,42217.0


Combine address fields into one column.

In [3]:
def combine_address_fields(row) -> str:
    """Concatenates address fields into one column/string value.
    
    Args:
        row: pandas dataframe row with 'Location Address', 'City', 'State Code', and 'Zip' columns

    Returns:
        str: full address

    """
    return f"{row['Location Address']} {row['City']} {row['State Code']} {row['Zip']}"

In [4]:
# apply function
df['full_address'] = df.apply(lambda row: combine_address_fields(row), axis=1)
df.full_address.sample(3)

233    500 CARTER RD OAK GROVE KY 42262.0
302             KY 244 RUSSELL KY 41169.0
112    INDUSTRY PARK RD FULTON KY 42041.0
Name: full_address, dtype: object

Geocode file using new column and geopy's RateLimiter, takes about 10 minutes.

In [5]:
# initialize new geocoder
geolocator = ArcGIS()
# pass locator to RateLimiter with delay
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# apply rate-limited geocoder to the full_address column, save result in location column
df['location'] = df['full_address'].apply(geocode)

Parse out location column into latitude and longitude columns.

In [6]:
df['latitude'] = df.location.apply(lambda x: x.latitude)
df['longitude'] = df.location.apply(lambda x: x.longitude)
df.drop(['location'], axis=1, inplace=True) # drop location column

In [18]:
# read shapefile for rest stops
shape = shapefile.Reader("../data/layer/RestAreaLocations_KYSZ.shp")

#first feature of the shapefile
features = shape.shapeRecords()

# make points
points = []
for feature in features.__geo_interface__['features']:
    point = Point(
        latitude=feature['properties']['Latitude'], 
        longitude=feature['properties']['Longitude']
    )
    points.append(point)

# use points to make new df with lat/long and concat to previous
rest_stops = pd.DataFrame.from_records([(p.latitude, p.longitude, 'Yes') for p in points], columns=('latitude', 'longitude', 'rest_stop'))
total = pd.concat([df, rest_stops])
total.reset_index(inplace=True, drop=True)

In [19]:
# measure distances for each point
def get_closest(df: pd.DataFrame) -> list[float]:
    indeces = []
    for i, row in df.iterrows():
        if row.rest_stop:
            target = Point(latitude=row.latitude, longitude=row.longitude)
            sample = df[df.rest_stop != 'Yes']
            others = [
                Point(latitude=lat, longitude=long) 
                for lat, long in zip(sample.latitude.values, sample.longitude.values)
            ]
            distances = [distance.distance(target, point) for point in others]
            closest = min(distances)
            index = distances.index(closest)
            indeces.append(index)
        else:
            indeces.append(None)
    return indeces

In [20]:
# apply above function
closest_indeces = get_closest(total)
total['closest'] = closest_indeces

In [26]:
# apply color column for closest to match point colors on eventually
def color(df: pd.DataFrame) -> list[str]:
    new_df = df.copy()
    new_df['color'] = '#FFFFFF'
    colors = px.colors.qualitative.Alphabet
    colors.append('#000000')
    
    temp = df[df.rest_stop == 'Yes'].reset_index()
    temp.columns = temp.columns[1:].insert(0, 'old_index')
    assert len(temp) == 27
    for i in range(27):
        # use this to color all 'close points' red
        #selected_color = px.colors.qualitative.Plotly[1]
        
        # use this to select color based on alphabaet pallete of 26
        selected_color = colors[i]
        
        temp.loc[i].color = selected_color
        old = temp.loc[i].old_index
        closest = temp.loc[i].closest
        new_df.loc[old, 'color'] = selected_color
        new_df.loc[closest, 'color'] = selected_color
    return new_df
            

In [27]:
# apply color function
colored = color(total)
colored.tail()

,Location Address,City,State Code,Zip,full_address,latitude,longitude,rest_stop,closest,color
409,NaN,NaN,NaN,NaN,NaN,38.203385,-85.329298,Yes,199,#FC1CBF
410,NaN,NaN,NaN,NaN,NaN,36.643445,-86.568690,Yes,119,#B00068
411,NaN,NaN,NaN,NaN,NaN,36.616147,-84.105513,Yes,128,#FBE426
412,NaN,NaN,NaN,NaN,NaN,38.168718,-84.766075,Yes,133,#FA0087
413,NaN,NaN,NaN,NaN,NaN,38.170222,-84.766152,Yes,133,#000000


## 4. Map Geocoded Addresses (Interactive)

In [28]:
# utilize plotly_express scatter_mapbox and dataframe columns
fig = px.scatter_mapbox(colored, lat="latitude", lon="longitude", hover_name="City" ,
                        color="color", zoom=5, height=800)
fig.update_layout(mapbox_style="open-street-map") # include streets
fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0}) # remove margins to make figure larger
fig.show()

In [14]:
shape = shapefile.Reader("../data/layer/RestAreaLocations_KYSZ.shp")

#first feature of the shapefile
features = shape.shapeRecords()

points = []
for feature in features.__geo_interface__['features']:
    point = geopy.point.Point(
        latitude=feature['properties']['Latitude'], 
        longitude=feature['properties']['Longitude']
    )
    points.append(point)

points

[Point(38.937132, -84.630865, 0.0),
 Point(38.938702, -84.633831, 0.0),
 Point(38.864814, -84.647903, 0.0),
 Point(37.946386, -85.690495, 0.0),
 Point(38.35642, -82.898557, 0.0),
 Point(38.352076, -82.912095, 0.0),
 Point(36.64963, -87.351527, 0.0),
 Point(38.020815, -84.140675, 0.0),
 Point(37.238489, -85.927053, 0.0),
 Point(37.238955, -85.929602, 0.0),
 Point(38.818981, -84.60123, 0.0),
 Point(37.042895, -84.097301, 0.0),
 Point(37.047545, -84.099337, 0.0),
 Point(37.050921, -88.652915, 0.0),
 Point(37.376437, -86.827028, 0.0),
 Point(38.338016, -85.516005, 0.0),
 Point(38.339607, -85.517966, 0.0),
 Point(37.796204, -83.704852, 0.0),
 Point(38.236439, -83.438447, 0.0),
 Point(38.230828, -83.443383, 0.0),
 Point(38.247873, -84.545895, 0.0),
 Point(38.249915, -84.548436, 0.0),
 Point(38.203385, -85.329298, 0.0),
 Point(36.643445, -86.56869, 0.0),
 Point(36.616147, -84.105513, 0.0),
 Point(38.168718, -84.766075, 0.0),
 Point(38.170222, -84.766152, 0.0)]

In [13]:
df.head()

,Location Address,City,State Code,Zip,full_address,latitude,longitude
0,2615 NEW MOODY LN,LA GRANGE,KY,40031.0,2615 NEW MOODY LN LA GRANGE KY 40031.0,38.389142,-85.391044
1,WASTEWATER DR,GREENVILLE,KY,42345.0,WASTEWATER DR GREENVILLE KY 42345.0,37.218246,-87.171780
2,530 ADAMS ST,MARION,KY,42064.0,530 ADAMS ST MARION KY 42064.0,37.344056,-88.063456
3,332 N FORK RD,HAZARD,KY,41701.0,332 N FORK RD HAZARD KY 41701.0,37.273824,-83.188699
4,280 NUGENT LN,HAWESVILLE,KY,42348.0,280 NUGENT LN HAWESVILLE KY 42348.0,37.906473,-86.763683


In [16]:
df.shape

(387, 7)

In [15]:
features.__geo_interface__['features']

[{'type': 'Feature',
  'properties': {'DESCRIPTIO': 'Boone County Rest Area Northbound I-75',
   'DISTRICT': 6,
   'Latitude': 38.937132,
   'Longitude': -84.630865,
   'Status': 'Open',
   'Facility_T': 'Rest Area'},
  'geometry': {'type': 'Point',
   'coordinates': (5239615.089983553, 4230882.141663477)}},
 {'type': 'Feature',
  'properties': {'DESCRIPTIO': 'Boone County Rest Area Southbound I-75',
   'DISTRICT': 6,
   'Latitude': 38.938702,
   'Longitude': -84.633831,
   'Status': 'Open',
   'Facility_T': 'Rest Area'},
  'geometry': {'type': 'Point',
   'coordinates': (5238764.543456301, 4231443.863965228)}},
 {'type': 'Feature',
  'properties': {'DESCRIPTIO': 'Boone County Trucker Rest Haven Southbound I-71',
   'DISTRICT': 6,
   'Latitude': 38.864814,
   'Longitude': -84.647903,
   'Status': 'Open',
   'Facility_T': 'Trucker Rest Haven'},
  'geometry': {'type': 'Point',
   'coordinates': (5235079.539015308, 4204485.138355225)}},
 {'type': 'Feature',
  'properties': {'DESCRIPTIO': 